**Mount Google Drive locally**

Mount your Google Drive on your runtime using an authorization code



In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
import cv2
import numpy as np
import keras
import numpy as np
import random
from imutils import paths
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.applications import vgg16
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers

dataset_path = "/content/drive/My Drive/Deep Learning with Python/Day 1/dataset"
IMAGE_DIMS = (224, 224, 3)
BS = 32
EPOCHS=100
print("keras version %s"%keras.__version__)
print("opencv version %s"%cv2.__version__)

# initialize the data and labels
data = []
labels = []

In [0]:
# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(dataset_path)))
random.seed(42)
random.shuffle(imagePaths)
# loop over the input images
for ind, imagePath in enumerate(imagePaths):
	# load the image, pre-process it, and store it in the data list
	print(ind, imagePath, imagePath.split(os.path.sep)[-2])
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)
 
	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

In [0]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(
	data.nbytes / (1024 * 1000.0)))

In [0]:
# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [0]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

In [0]:
# initialize the model
print("[INFO] compiling model...")

# Load VGG16 model without the top layers
base_layers = vgg16.VGG16(include_top=False, input_shape=IMAGE_DIMS)
#base_layers = vgg16.VGG16(include_top=False)

# Allow fine tuning to go into the convolution layers
for layer in base_layers.layers:
     layer.trainable = False

# Create the network
network = models.Sequential(base_layers.layers)
network.add(layers.Flatten())
network.add(layers.Dense(5, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.summary()

In [0]:
history = network.fit(trainX, trainY, epochs=50, batch_size=32, verbose=2)
test_loss, test_acc = network.evaluate(testX, testY, verbose=2)
print('test_acc:', test_acc)

Load an image for testing

In [0]:
# load the image
# image = cv2.imread(dataset_path + '/mewtwo/00000000.jpg')
image = cv2.imread(dataset_path + '/pikachu/00000000.jpg')

# pre-process the image for classification
image = cv2.resize(image, (IMAGE_DIMS[0], IMAGE_DIMS[1]))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

In [0]:
# classify the input image
print("[INFO] classifying image...")
proba = network.predict(image)[0]
idx = np.argmax(proba)
label = lb.classes_[idx]

print(label)